In [1]:
import torch
import onnx
import onnxruntime
import numpy as np
import transformers
from gpt3_onnx.generation_onnx import GPT2BeamSearchGenerator, GPT2Generator, GPT2BeamSearchSampleGenerator
from transformers import GPT2Tokenizer
import os


print(torch.__version__)
print(onnx.__version__)
print(onnxruntime.__version__)
print(transformers.__version__)

1.10.2+cpu
1.10.2
1.10.0
4.11.3


In [2]:
num_beams = 4
max_length=20

In [3]:
chk_path = 'C:/Users/edbon/devproj/faiky-tails/savedir/s_kw/checkpoints/'

with open(f'{chk_path}checkpoint.pt', 'rb') as f:
    model = torch.load(f, map_location='cpu')

tokenizer = GPT2Tokenizer.from_pretrained('C:/Users/edbon/devproj/faiky-tails/savedir/tokenizer')
# text_encoder.pad_token = text_encoder.eos_token

PROMT = ["старик", "Кощей"]

padtok =  0
septok = tokenizer.convert_tokens_to_ids('[SEP]')
starttok = tokenizer.convert_tokens_to_ids('<s>')
endtok = tokenizer.convert_tokens_to_ids('</s>')


model_config_custom = {
    "bos_token_id": starttok,
    "eos_token_id": endtok,
    "decoder_start_token_id": septok,
    "pad_token_id": padtok,
    "forced_bos_token_id": None,
    "no_repeat_ngram_size": 0,
    "min_length": 0
}

model.config.update(model_config_custom)

endkeytok = tokenizer.convert_tokens_to_ids('_endkw_')

context = " _kw_ ".join(PROMT)
context = tokenizer.encode(context)

context = [starttok] + context + [endkeytok] + [septok]
context = tokenizer.decode(context)

inputs = tokenizer(context, max_length=max_length, truncation=True, return_tensors="pt").to(model.device)

model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


# Beam Search

In [4]:
summary_ids = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    num_beams=num_beams,
    max_length=max_length,
    early_stopping=True,
    decoder_start_token_id=septok,
    pad_token_id=padtok,
    bos_token_id=starttok,
    eos_token_id=endtok,
    forced_eos_token_id=endtok,
)

In [ ]:
ort_sess = None
script_model = torch.jit.script(GPT2BeamSearchGenerator(model))

In [ ]:
script_model.graph

In [7]:
onnx_file_path = 'gpt3_beam.onnx'

In [8]:
with torch.no_grad():
        PROMT = ["старик", "Кощей"]

        context = " _kw_ ".join(PROMT)
        context = tokenizer.encode(context)
        endkeytok = tokenizer.convert_tokens_to_ids('_endkw_')

        context = [starttok] + context + [endkeytok] + [septok]
        context = tokenizer.decode(context)

        inputs = tokenizer(context, max_length=model.config.n_ctx, return_tensors="pt").to(model.device)
        

        torch.onnx.export(
            script_model,
            (
                inputs["input_ids"],
                inputs["attention_mask"],
                num_beams,
                max_length,
            ),
            onnx_file_path,
            opset_version=14,  # TODO for gpt maybe 12?
            input_names=["input_ids", "attention_mask", "num_beams", "max_length"], 
            output_names=["output_ids"],
            dynamic_axes={
                "input_ids": {0: "batch", 1: "seq"},
                "output_ids": {0: "batch", 1: "seq_out"},
            },
            example_outputs=summary_ids,
            verbose=True
        )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\edbon\anaconda3\envs\tf\lib\site-packages\torch\onnx\utils.py:100: UserWarning: `example_outputs' is deprecated and ignored. Will be removed in next PyTorch release.
  warnings.warn("`example_outputs' is deprecated and ignored. Will be removed in "
C:\Users\edbon\anaconda3\envs\tf\lib\site-packages\torch\onnx\symbolic_helper.py:716: UserWarning: allowzero=0 by default. In order to honor zero value in shape use allowzero=1
  warnings.warn("allowzero=0 by default. In order to honor zero value in shape use allowzero=1")


graph(%input_ids : Long(*, *, strides=[8, 1], requires_grad=0, device=cpu),
      %attention_mask : Long(1, 8, strides=[8, 1], requires_grad=0, device=cpu),
      %num_beams : Long(device=cpu),
      %max_length : Long(device=cpu),
      %decoder_no_past.decoder.transformer.wte.weight : Float(50260, 768, strides=[768, 1], requires_grad=0, device=cpu),
      %decoder_no_past.decoder.transformer.wpe.weight : Float(2048, 768, strides=[768, 1], requires_grad=0, device=cpu),
      %decoder_no_past.decoder.transformer.h.0.ln_1.bias : Float(768, strides=[1], requires_grad=0, device=cpu),
      %decoder_no_past.decoder.transformer.h.0.ln_1.weight : Float(768, strides=[1], requires_grad=0, device=cpu),
      %decoder_no_past.decoder.transformer.h.0.attn.masked_bias : Float(requires_grad=0, device=cpu),
      %decoder_no_past.decoder.transformer.h.0.attn.bias : Byte(1, 1, 2048, 2048, strides=[4194304, 4194304, 2048, 1], requires_grad=0, device=cpu),
      %decoder_no_past.decoder.transformer.h.0

In [9]:
import onnx

# Load the ONNX model
model_onnx_beam = onnx.load(onnx_file_path)

# Check that the model is well formed
onnx.checker.check_model(model_onnx_beam)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model_onnx_beam.graph))

graph torch-jit-export (
  %input_ids[INT64, batchxseq]
  %attention_mask[INT64, 1x8]
  %num_beams[INT64, scalar]
  %max_length[INT64, scalar]
) initializers (
  %decoder_no_past.decoder.transformer.wte.weight[FLOAT, 50260x768]
  %decoder_no_past.decoder.transformer.wpe.weight[FLOAT, 2048x768]
  %decoder_no_past.decoder.transformer.h.0.ln_1.bias[FLOAT, 768]
  %decoder_no_past.decoder.transformer.h.0.ln_1.weight[FLOAT, 768]
  %decoder_no_past.decoder.transformer.h.0.attn.masked_bias[FLOAT, scalar]
  %decoder_no_past.decoder.transformer.h.0.attn.bias[UINT8, 1x1x2048x2048]
  %decoder_no_past.decoder.transformer.h.0.attn.c_attn.weight[FLOAT, 768x2304]
  %decoder_no_past.decoder.transformer.h.0.attn.c_attn.bias[FLOAT, 2304]
  %decoder_no_past.decoder.transformer.h.0.attn.c_proj.weight[FLOAT, 768x768]
  %decoder_no_past.decoder.transformer.h.0.attn.c_proj.bias[FLOAT, 768]
  %decoder_no_past.decoder.transformer.h.0.ln_2.bias[FLOAT, 768]
  %decoder_no_past.decoder.transformer.h.0.ln_2.weight[F

In [10]:
import onnxruntime
import numpy as np

options = onnxruntime.SessionOptions()

ort_sess = onnxruntime.InferenceSession(onnx_file_path, sess_options=options )

In [11]:
ort_inputs = {
        "input_ids": inputs["input_ids"].cpu().numpy(),
        "attention_mask": inputs["attention_mask"].cpu().numpy(),
        "num_beams": np.array(num_beams, dtype=np.int64),
        "max_length": np.array(max_length, dtype=np.int64),
    }

ort_out = ort_sess.run(
    None,
    ort_inputs
)

print(summary_ids)
print(ort_out)

tensor([[    1,   419,  2611, 50258, 18989,  2909, 50259, 50257,   365, 28715,
          3638,  1207,    16,   282, 28715, 23940,  3942,    17, 18047,     2]])
[array([[    1,   419,  2611, 50258, 18989,  2909, 50259, 50257,   365,
        28715,  3638,  1207,    16,   282, 28715, 23940,  3942,    17,
        18047,     2]], dtype=int64)]


In [12]:
np.testing.assert_allclose(summary_ids.cpu().numpy(), ort_out[0], rtol=1e-3, atol=1e-3)

# BeamsSearchSample

In [27]:
temperature=2.0
top_p=0.95

In [28]:
summary_ids_temp = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    num_beams=num_beams,
    max_length=max_length,
    early_stopping=True,
    decoder_start_token_id=septok,
    pad_token_id=padtok,
    bos_token_id=starttok,
    eos_token_id=endtok,
    forced_eos_token_id=endtok,
    temperature=temperature,
    top_p=top_p
)

In [29]:
ort_sess = None
script_model_sample = torch.jit.script(GPT2BeamSearchSampleGenerator(model))

In [30]:
onnx_file_path = 'gpt3_sample.onnx'

In [31]:
print(summary_ids_temp)

tensor([[    1,   419,  2611, 50258, 18989,  2909, 50259, 50257,   365, 28715,
          3638,  1207,    16,   282, 28715, 23940,  3942,    17, 18047,     2]])


In [32]:
with torch.no_grad():
        PROMT = ["старик", "Кощей"]

        context = " _kw_ ".join(PROMT)
        context = tokenizer.encode(context)
        endkeytok = tokenizer.convert_tokens_to_ids('_endkw_')

        context = [starttok] + context + [endkeytok] + [septok]
        context = tokenizer.decode(context)

        inputs = tokenizer(context, max_length=model.config.n_ctx, return_tensors="pt").to(model.device)
        
        torch.onnx.export(
            script_model_sample,
            (
                inputs["input_ids"],
                inputs["attention_mask"],
                num_beams,
                max_length,
                temperature,
                top_p
            ),
            onnx_file_path,
            opset_version=14,  # TODO for gpt maybe 12?
            input_names=["input_ids", "attention_mask", "num_beams", "max_length", "temperature","top_p"], 
            output_names=["output_ids"],
            dynamic_axes={
                "input_ids": {0: "batch", 1: "seq"},
                "output_ids": {0: "batch", 1: "seq_out"},
            },
            example_outputs=summary_ids_temp,
            verbose=True
        )

graph(%input_ids : Long(*, *, strides=[8, 1], requires_grad=0, device=cpu),
      %attention_mask : Long(1, 8, strides=[8, 1], requires_grad=0, device=cpu),
      %num_beams : Long(device=cpu),
      %max_length : Long(device=cpu),
      %temperature : Double(requires_grad=0, device=cpu),
      %top_p : Double(requires_grad=0, device=cpu),
      %decoder_no_past.decoder.transformer.wte.weight : Float(50260, 768, strides=[768, 1], requires_grad=0, device=cpu),
      %decoder_no_past.decoder.transformer.wpe.weight : Float(2048, 768, strides=[768, 1], requires_grad=0, device=cpu),
      %decoder_no_past.decoder.transformer.h.0.ln_1.bias : Float(768, strides=[1], requires_grad=0, device=cpu),
      %decoder_no_past.decoder.transformer.h.0.ln_1.weight : Float(768, strides=[1], requires_grad=0, device=cpu),
      %decoder_no_past.decoder.transformer.h.0.attn.masked_bias : Float(requires_grad=0, device=cpu),
      %decoder_no_past.decoder.transformer.h.0.attn.bias : Byte(1, 1, 2048, 2048, str

In [33]:
# Load the ONNX model
model_onnx_sample = onnx.load(onnx_file_path)

# Check that the model is well formed
onnx.checker.check_model(model_onnx_sample)

In [34]:
# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model_onnx_sample.graph))

graph torch-jit-export (
  %input_ids[INT64, batchxseq]
  %attention_mask[INT64, 1x8]
  %num_beams[INT64, scalar]
  %max_length[INT64, scalar]
  %temperature[DOUBLE, scalar]
  %top_p[DOUBLE, scalar]
) initializers (
  %decoder_no_past.decoder.transformer.wte.weight[FLOAT, 50260x768]
  %decoder_no_past.decoder.transformer.wpe.weight[FLOAT, 2048x768]
  %decoder_no_past.decoder.transformer.h.0.ln_1.bias[FLOAT, 768]
  %decoder_no_past.decoder.transformer.h.0.ln_1.weight[FLOAT, 768]
  %decoder_no_past.decoder.transformer.h.0.attn.masked_bias[FLOAT, scalar]
  %decoder_no_past.decoder.transformer.h.0.attn.bias[UINT8, 1x1x2048x2048]
  %decoder_no_past.decoder.transformer.h.0.attn.c_attn.weight[FLOAT, 768x2304]
  %decoder_no_past.decoder.transformer.h.0.attn.c_attn.bias[FLOAT, 2304]
  %decoder_no_past.decoder.transformer.h.0.attn.c_proj.weight[FLOAT, 768x768]
  %decoder_no_past.decoder.transformer.h.0.attn.c_proj.bias[FLOAT, 768]
  %decoder_no_past.decoder.transformer.h.0.ln_2.bias[FLOAT, 768]


In [35]:
options = onnxruntime.SessionOptions()

ort_sess = onnxruntime.InferenceSession(onnx_file_path, sess_options=options )

In [36]:
ort_inputs = {
        "input_ids": inputs["input_ids"].cpu().numpy(),
        "attention_mask": inputs["attention_mask"].cpu().numpy(),
        "num_beams": np.array(num_beams, dtype=np.int64),
        "max_length": np.array(max_length, dtype=np.int64),
        "temperature": np.array(temperature, dtype=np.float64),
        "top_p": np.array(top_p, dtype=np.float64),
    }

ort_out = ort_sess.run(
    None,
    ort_inputs
)

In [37]:
print(summary_ids)
print(summary_ids_temp)
print(ort_out)

tensor([[    1,   419,  2611, 50258, 18989,  2909, 50259, 50257,   365, 28715,
          3638,  1207,    16,   282, 28715, 23940,  3942,    17, 18047,     2]])
tensor([[    1,   419,  2611, 50258, 18989,  2909, 50259, 50257,   365, 28715,
          3638,  1207,    16,   282, 28715, 23940,  3942,    17, 18047,     2]])
[array([[    1,   419,  2611, 50258, 18989,  2909, 50259, 50257,  1681,
          808,   417,  1300,   945,   811,   329,   417,   418,  3209,
          411,     2]], dtype=int64)]


In [38]:
print("hg:", tokenizer.decode(summary_ids_temp[0], skip_special_tokens=False, clean_up_tokenization_spaces=False))
print("onnx:",tokenizer.decode(ort_out[0][0], skip_special_tokens=False, clean_up_tokenization_spaces=False))

hg: <s>старик _kw_ Кощей _endkw_ [SEP]  В некотором царстве, в некотором государстве жил-был</s>
onnx: <s>старик _kw_ Кощей _endkw_ [SEP]  Под время я запрудился у явочек из</s>


In [ ]:
np.testing.assert_allclose(summary_ids.cpu().numpy(), ort_out[0], rtol=1e-3, atol=1e-3)